# Uncertainty-Aware Road Obstacle Identification

## 1. Imports
Import necessary libraries like PyTorch, torchvision, etc.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import Cityscapes     # ready-made dataset wrapper
import numpy as np
import matplotlib.pyplot as plt
import pathlib, shutil, zipfile
from tqdm import tqdm # Import tqdm
from torch.utils.data import Subset

## 2. Globals
Define global variables such as paths, batch size, learning rate, etc.

In [2]:
# Global variables
DATASET_PATH = "./datasets/"
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS = 50

# Select the best available device (CPU, CUDA, or MPS (Metal Performance Shaders for macOS))
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using CUDA device: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Metal Performance Shaders) device")
else:
    device = torch.device("cpu")
    print("Using CPU")
DEVICE = device

Using CPU


## 3. Utils
Helper functions for visualization, preprocessing, and more.

In [3]:

def visualize_sample(image, mask):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Image")
    plt.imshow(image)
    plt.subplot(1, 2, 2)
    plt.title("Mask")
    plt.imshow(mask)
    plt.show()


In [4]:
def unzip(zip_path,dest):
  print("Extracting {zippath.name} ...")
  with zipfile.ZipFile(zip_path) as z:
    z.extractall(dest)
  print("done!")

### 3.1 Utils for Conformal Risk Control

Miscoverage Loss

In [5]:
def miscoverage_loss(set_mask: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
    """
    Calculate the miscoverage loss for a set prediction.

    Args:
        set_mask: A binary tensor of shape [C, H, W] where set_mask[k,i,j] == 1
                  if class k is in the predicted set for pixel (i,j).
        labels: A tensor of shape [H, W] with ground-truth class IDs.

    Returns:
        A scalar tensor representing the miscoverage rate (proportion of
        pixels where the true label is not in the predicted set),
        ignoring pixels with invalid labels.
    """
    C, H, W = set_mask.shape
    # Create a mask for valid labels (class IDs within the range [0, C-1])
    valid_pixel_mask = (labels >= 0) & (labels < C)

    # Create a modified labels tensor where invalid labels are temporarily set to a valid index (e.g., 0)
    # This prevents scatter_ from failing on invalid indices like 255.
    # Ensure the temporary index is within the valid range [0, C-1]. 0 is a safe choice if it's always a valid class ID.
    # If 0 is not guaranteed to be a valid class ID in the dataset, you might need a different placeholder,
    # but for Cityscapes 0 ('road') is usually valid.
    placeholder_label = 0 # A valid class index to use temporarily
    masked_labels = torch.where(valid_pixel_mask, labels, torch.tensor(placeholder_label, device=labels.device, dtype=labels.dtype))

    # Create a ground-truth mask using the masked labels
    gt_mask = torch.zeros_like(set_mask)
    # Scatter values using the masked_labels tensor
    gt_mask.scatter_(0, masked_labels.unsqueeze(0), 1)     # in-place one-hot

    # Pixel is covered if gt class ∈ predicted set
    # This calculation includes pixels with invalid labels temporarily set to 0 in gt_mask
    covered = (set_mask * gt_mask).sum(dim=0) # covered is [H, W], 1 if true class in set, 0 otherwise

    # Now, apply the valid_pixel_mask to exclude contributions from invalid pixels
    covered = covered * valid_pixel_mask.float()

    # Calculate total number of valid pixels
    num_valid_pixels = valid_pixel_mask.sum().float()

    if num_valid_pixels == 0:
        # If there are no valid pixels, the miscoverage is undefined or can be considered 0.0
        # Returning 0.0 is a reasonable default in this context.
        return torch.tensor(0.0, device=set_mask.device)

    # Calculate coverage ratio only over valid pixels
    coverage = covered.sum() / num_valid_pixels

    # Miscoverage is 1 - coverage
    return 1.0 - coverage


Least Ambiguous Set-Valued Classifiers

In [6]:
def T_lambda(probs, lam):
    # probs: [C, H, W], lam: float ∈ [0,1]
    return (probs >= (1 - lam)).float()  # binariza por canal

Dichotomic search over the parameter λ

In [7]:

"""
We can compute the optimal like this because the Loss is monotonic in lambda
-  Lambda is the threshold for binarization in T_lambda
-  Alpha is the tolerated risk level
-  Eps is the precision for the binary search
"""

def binary_search_lambda(probs_list,labels_list,alpha=0.1,eps=1e-3):
    left , right = 0.0, 1.0
    while right - left > eps:
        mid = (left + right) / 2.0
        print(mid)

        risk_sum = 0.0
        for probs,labels in zip(probs_list,labels_list):
            mask = T_lambda(probs, mid)
            risk_sum += miscoverage_loss(mask,labels)
        avr_risk += risk_sum / len(probs_list)
        if avr_risk <= alpha:
            right = mid
        else:
            left = mid
    return mid

In [8]:
def online_risk_lambda(lam,loader):
  "We calculate the risk online so the RAM does not fills up"
  risk_sum, n = 0.0,0.0
  with torch.no_grad():
    for img, gt in tqdm(loader, desc=f"Calculating risk for lambda={lam:.4f}"):
      img = img.to(device)
      gt = gt.squeeze(0)
      out = model(img)['out'][0]
      probs = torch.softmax(out,dim=0).cpu()
      mask = T_lambda(probs,lam)
      risk_sum += miscoverage_loss(mask,gt)
      n +=1
      del img, out, probs,mask
      torch.cuda.empty_cache()
    avg_risk = risk_sum / n
    return avg_risk

In [9]:
def online_binary_search(model, loader, alpha=0.1, eps=1e-3, device='cuda'):
    # Búsqueda binaria
    left, right = 0.0, 1.0
    while right - left > eps:
        mid   = (left + right) / 2
        r_mid = online_risk_lambda(mid,loader)
        if r_mid <= alpha:
            right = mid
        else:
            left  = mid
    return (left + right) / 2

## 4. Data
Load and preprocess datasets like Cityscapes, LostAndFound, and Fishyscapes.

In [10]:
# Code for data loading and preprocessing

In [11]:
#@title Retrieving CityScapes from Drive
# Mount Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Path where the two ZIPs are uploaded
GDRIVE_CS = pathlib.Path("/content/drive/MyDrive/datasets/cityscapes")

assert (GDRIVE_CS / 'leftImg8bit_trainvaltest.zip').exists(), "Missing ZIP in Drive."
assert (GDRIVE_CS / 'gtFine_trainvaltest.zip').exists(), "Missing ZIP in Drive."
print("ZIPs found in Drive")

Mounted at /content/drive
ZIPs found in Drive


In [12]:
#@title Copying and extracting to VM
#Lets copy them to the VM (if they are not already there)
COLAB_CS = pathlib.Path('/content/cityscapes')
assert not (COLAB_CS / 'leftImg8bit_trainvaltest.zip').exists(), "Zip file already in VM."
assert not (COLAB_CS / 'gtFine_trainvaltest.zip').exists(), "Zip file already in VM."

COLAB_CS.mkdir(parents=True, exist_ok=True)
shutil.copy(GDRIVE_CS / 'leftImg8bit_trainvaltest.zip', '/content/')
shutil.copy(GDRIVE_CS / 'gtFine_trainvaltest.zip', '/content/')

print("ZIP files copied from GDrive")

for zname in ('leftImg8bit_trainvaltest.zip','gtFine_trainvaltest.zip'):
  zippath = pathlib.Path('/content') / zname
  unzip(zippath, COLAB_CS)
  zippath.unlink()  #delete zip file
  print("ZIP files extracted and deleted")


ZIP files copied from GDrive
Extracting {zippath.name} ...
done!
ZIP files extracted and deleted
Extracting {zippath.name} ...
done!
ZIP files extracted and deleted


In [15]:
#@title Dataset + DataLoader

# Convert the RGB PIL image (H,W,3 uint8) into a float32 tensor in [0,1]
img_tf = transforms.ToTensor()
# Ground-truth masks come in as PIL “mode =L” images (H,W uint8). We need them as int64 tensors for loss/calibration, so:
# lbl_tf = transforms.Lambda(
#     lambda pil_img: torch.as_tensor(np.array(pil_img), dtype=torch.long)
# )

# 1. First define the label mapping (34 → 19 classes)
# This is the official Cityscapes mapping:
cityscapes_train_ids = {
    0: 255, 1: 255, 2: 255, 3: 255, 4: 255, 5: 255, 6: 255,
    7: 0, 8: 1, 9: 255, 10: 255, 11: 2, 12: 3, 13: 4,
    14: 255, 15: 255, 16: 255, 17: 5, 18: 255, 19: 6,
    20: 7, 21: 8, 22: 9, 23: 10, 24: 11, 25: 12, 26: 13,
    27: 14, 28: 15, 29: 255, 30: 255, 31: 16, 32: 17, 33: 18
}

def convert_to_train_ids(pil_img):
    # Convert PIL to numpy array
    labels = np.array(pil_img)
    # Vectorized mapping using numpy
    train_ids = np.vectorize(cityscapes_train_ids.get)(labels)
    return torch.as_tensor(train_ids, dtype=torch.long)


# ready-made dataset wrapper
val_set = Cityscapes(
    root=str(COLAB_CS),
    split = 'val',            # 500 fine labelled images
    mode='fine',              # using the accurate (not coarse) masks
    target_type ='semantic',  # we want a single channel with class-IDs
    transform=img_tf,         # apply to the RGB image
    # target_transform=lbl_tf   # apply to the mask
    target_transform = convert_to_train_ids
)

#Randomly select 100 indices
num_calibration = 50
indices = torch.randperm(len(val_set))[:num_calibration]

#Create Subset
calibration_set = Subset(val_set,indices)
# Verify
print(f"Calibration set size: {len(calibration_set)}")  # Should be 100

#Wraping in dataloader
calibration_loader = DataLoader(
    calibration_set,
    batch_size=1,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

val_loader =torch.utils.data.DataLoader(
    val_set,
    batch_size=1,            # one image per step
    shuffle=False,           # no randomise
    num_workers=2,           # 2 background workers decode PNGs in parallel
    pin_memory=True          # speed-up host→GPU transfer when using CUDA
)

#Sanity Check
assert len(val_set) == 500, "Sanity Check  unsuccesful. Val set should have 500 items"
sample = val_set[0]
print("Unique labels:", torch.unique(sample[1]))  # Should show 0-18 + 255

Calibration set size: 50
Unique labels: tensor([  0,   1,   2,   5,   6,   7,   8,  10,  11,  12,  13,  17,  18, 255])


## 5. Network
Define the neural network architecture for semantic segmentation.

In [16]:
# Define the model architecture here


# ------------------------------------------------------------------
# Load a pretrained DeepLabV3 with ResNet-50 backbone
# ------------------------------------------------------------------
model = models.segmentation.deeplabv3_resnet50(pretrained=False, num_classes=19).to(device)                         # move network to the selected device
model.eval()                                 # inference mode (deactivate dropout/BN updates)



/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 152MB/s]


DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

### Conformal Risk Control

#### Implementation
1. Pass the test set through the model to get logits and compute softmax
2. Use optimal Lambda to get mask
3. Visualize the cardinality of the set. (Pixel wise anomaly detection measurement)


In [17]:
lambda_hat = online_binary_search(model, calibration_loader, alpha=0.9)
print(f"λ̂ obtenido: {lambda_hat:.4f}")

Calculating risk for lambda=0.7500:  88%|████████▊ | 44/50 [46:23<06:19, 63.26s/it]


KeyboardInterrupt: 

In [ ]:
# #@title Calibration
# # 1. Pass the calibration set through the model to get logits
# # 2. Convert logits to probabilities using softmax.
# # 3. Run binary_search_lambda over the calibration probabilities to find the optimal Lambda


# probs_list, labels_list = [],[]

# with torch.no_grad():
#       for img, gt in val_loader:
#         img = img.to(device)                 # move image tensor to GPU/CPU
#         logits = model(img)['out']           # dict → key 'out'; shape [1, C, H', W']
#         # Convert logits to class-probabilities along channel dimension C
#         probs = torch.softmax(logits[0], dim=0).cpu()  # remove batch dim
#         # ----------- free everything that lives on GPU -----------
#         del logits, img                         # drop GPU tensors
#         torch.cuda.empty_cache()                # give the arena back to CUDA
#         # ----------------------------------------------------------
#         probs_list.append(probs)             # store for calibration / evaluation
#         labels_list.append(gt.squeeze(0))    # remove batch dim → shape [H, W]

# # At this point:
# # probs_list[i] → torch.Tensor of shape [C, H, W] with per-class probabilities.
# # labels_list[i] → torch.LongTensor shape [H, W] with ground-truth IDs.

In [ ]:
# lambda_hat = binary_search_lambda(probs_list,labels_list,0.1,eps=1e-3)
# # print(f"λ̂ = {lambda_hat:.6f}")


In [ ]:
# from tqdm import tqdm  # tqdm adds a smart progress-bar to any Python loop

# # ----------------------------------------------------------
# # Evaluate a loader with a *fixed* λ̂
# # ----------------------------------------------------------
# def evaluate(loader, lambda_hat):
#     """
#     Prints the mean risk; should be ≤ alpha if the same split was
#     used for calibration. If you run it on a fresh test set, this is
#     an unbiased estimate of the true risk.
#     """
#     risk_sum, n = 0.0, 0          # accumulate loss & sample count

#     with torch.no_grad():
#         # tqdm(...) wraps any iterator and shows a live progress-bar
#         for img, gt in tqdm(loader, desc="Eval"):
#             img = img.to(device)                          # move RGB to GPU
#             logits = model(img)['out']                    # forward pass
#             probs  = torch.softmax(logits[0], dim=0).cpu()# C×H×W  ➜ CPU
#             mask   = T_lambda(probs, lambda_hat)          # prediction set S_λ(x)

#             # Your monotone loss (scalar tensor)
#             risk   = miscoverage_loss(mask, gt.squeeze(0))
#             risk_sum += risk.item()
#             n += 1

#     avg_risk = risk_sum / n
#     print(f"Average risk on evaluation split: {avg_risk:.4f}  (target α={alpha})")


## 6. Train
Implement the training loop, including loss functions and optimizers.

In [ ]:
# Training loop and loss computation

## 7. Test
Test the trained model and evaluate performance metrics.

In [ ]:
# Evaluation metrics and performance analysis

In [16]:
import numpy as np
import torch

unique_class_ids = set()

# Iterate through the validation dataset
for _, labels in val_set:
    # Convert the labels tensor to a NumPy array and find unique values
    unique_ids = np.unique(labels.numpy())
    # Add the unique IDs to the set
    unique_class_ids.update(unique_ids)

# Convert the set to a sorted list for better readability
sorted_unique_class_ids = sorted(list(unique_class_ids))

print("Unique class IDs found in the validation set:")
print(sorted_unique_class_ids)

Unique class IDs found in the validation set:
[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33)]


In [ ]:
import torch
from torchvision.transforms import InterpolationMode # Make sure this is imported if needed for previous steps
from tqdm import tqdm # Import tqdm

# ----------------------------------------------------------
# Monotone loss function (miscoverage indicator)
# ----------------------------------------------------------

def miscoverage_loss(set_mask: torch.Tensor, labels: torch.Tensor) -> torch.Tensor:
    """
    Calculate the miscoverage loss for a set prediction.

    Args:
        set_mask: A binary tensor of shape [C, H, W] where set_mask[k,i,j] == 1
                  if class k is in the predicted set for pixel (i,j).
        labels: A tensor of shape [H, W] with ground-truth class IDs.

    Returns:
        A scalar tensor representing the miscoverage rate (proportion of
        pixels where the true label is not in the predicted set),
        ignoring pixels with invalid labels.
    """
    C, H, W = set_mask.shape
    # Create a mask for valid labels (class IDs within the range [0, C-1])
    valid_pixel_mask = (labels >= 0) & (labels < C)

    # Create a modified labels tensor where invalid labels are temporarily set to a valid index (e.g., 0)
    # This prevents scatter_ from failing on invalid indices like 255.
    # Ensure the temporary index is within the valid range [0, C-1]. 0 is a safe choice if it's always a valid class ID.
    # If 0 is not guaranteed to be a valid class ID in the dataset, you might need a different placeholder,
    # but for Cityscapes 0 ('road') is usually valid.
    placeholder_label = 0 # A valid class index to use temporarily
    masked_labels = torch.where(valid_pixel_mask, labels, torch.tensor(placeholder_label, device=labels.device, dtype=labels.dtype))

    # Create a ground-truth mask using the masked labels
    gt_mask = torch.zeros_like(set_mask)
    # Scatter values using the masked_labels tensor
    gt_mask.scatter_(0, masked_labels.unsqueeze(0), 1)     # in-place one-hot

    # Pixel is covered if gt class ∈ predicted set
    # This calculation includes pixels with invalid labels temporarily set to 0 in gt_mask
    covered = (set_mask * gt_mask).sum(dim=0) # covered is [H, W], 1 if true class in set, 0 otherwise

    # Now, apply the valid_pixel_mask to exclude contributions from invalid pixels
    covered = covered * valid_pixel_mask.float()

    # Calculate total number of valid pixels
    num_valid_pixels = valid_pixel_mask.sum().float()

    if num_valid_pixels == 0:
        # If there are no valid pixels, the miscoverage is undefined or can be considered 0.0
        # Returning 0.0 is a reasonable default in this context.
        return torch.tensor(0.0, device=set_mask.device)

    # Calculate coverage ratio only over valid pixels
    coverage = covered.sum() / num_valid_pixels

    # Miscoverage is 1 - coverage
    return 1.0 - coverage

# ----------------------------------------------------------
# Set-valued prediction (thresholding function)
# ----------------------------------------------------------

def T_lambda(probs: torch.Tensor, lam: float) -> torch.Tensor:
    """
    Construct a set-valued prediction S(x) by thresholding the probability
    vector p(x).

    Args:
        probs: A tensor of shape [C, H, W] with per-class probabilities.
        lam: The threshold parameter λ.

    Returns:
        A binary tensor of shape [C, H, W] where S[k,i,j] == 1 if
        class k is included in the predicted set for pixel (i,j).
    """
    # S_λ(x) = {k : p_k(x) ≥ λ}
    return (probs >= lam).to(torch.uint8)


# ----------------------------------------------------------
# Risk function R(λ)
# ----------------------------------------------------------

def online_risk_lambda(lam: float, loader: torch.utils.data.DataLoader, model: torch.nn.Module, device='cuda') -> float:
    """
    Calculate the average miscoverage risk for a given lambda on a dataset
    loaded by `loader`.
    """
    risk_sum, n = 0.0, 0          # accumulate loss & sample count

    # Ensure model is in evaluation mode
    model.eval()

    with torch.no_grad():
        # Wrap the loader with tqdm for a progress bar
        for img, gt in tqdm(loader, desc=f"Calculating risk for lambda={lam:.4f}"):
            img = img.to(device)                 # move image tensor to GPU/CPU
            # Ensure gt is on the same device as labels will be processed
            # gt = gt.to(device) # Keep gt on CPU as miscoverage_loss expects CPU tensor for labels
            gt = gt.squeeze(0) # Remove batch dimension from ground truth

            out = model(img)['out']           # dict → key 'out'; shape [1, C, H', W']
            probs = torch.softmax(out[0],dim=0).cpu() # Move probabilities to CPU to save GPU memory
            mask = T_lambda(probs,lam)

            # Pass gt (on CPU) and mask (on CPU) to miscoverage_loss
            risk_sum += miscoverage_loss(mask, gt).item() # Use .item() to get scalar value
            n +=1

            # Explicitly delete tensors and clear cache to manage memory
            del img, out, probs, mask, gt
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    if n == 0:
        return 0.0 # Return 0 risk if loader is empty

    return risk_sum / n


# ----------------------------------------------------------
# Binary search for the optimal lambda
# ----------------------------------------------------------
def online_binary_search(model: torch.nn.Module, loader: torch.utils.data.DataLoader, alpha: float, eps: float = 1e-3, device='cuda') -> float:
    """
    Find the optimal lambda using binary search such that R(lambda) <= alpha.
    """
    left, right = 0.0, 1.0 # Lambda is a probability, so it's between 0 and 1

    print(f"Starting binary search for lambda between {left} and {right} with target alpha={alpha} and eps={eps}")

    # Initial check for bounds (optional but good practice)
    # Be cautious with calling online_risk_lambda multiple times if it's computationally expensive
    # risk_at_0 = online_risk_lambda(left, loader, model, device)
    # if risk_at_0 <= alpha:
    #     print(f"Risk at lambda=0 ({risk_at_0:.4f}) is <= alpha ({alpha}). Optimal lambda is 0.")
    #     return left

    # risk_at_1 = online_risk_lambda(right, loader, model, device)
    # if risk_at_1 > alpha:
    #      print(f"Warning: Even with lambda=1, risk ({risk_at_1:.4f}) is > alpha ({alpha}). Consider increasing alpha or checking model/data.")
         # The binary search will converge to 1.0 in this case.


    iteration = 0
    while right - left > eps:
        mid   = (left + right) / 2
        # print(f"Iteration {iteration}: Checking lambda = {mid:.4f}") # Debug print
        r_mid = online_risk_lambda(mid,loader, model, device)
        print(f"Iteration {iteration}: Lambda = {mid:.4f}, Risk = {r_mid:.4f}") # Progress update

        if r_mid <= alpha:
            right = mid
        else:
            left = mid
        iteration += 1

    # The optimal lambda is typically the largest lambda such that R(lambda) <= alpha.
    # Due to the nature of binary search and the monotonicity, the right bound
    # will be the largest value found that satisfies the condition (within epsilon).
    print(f"Binary search finished. Optimal lambda found: {right:.6f}")
    return right

# Now you can call online_binary_search with your model, val_loader, and desired alpha
# For example:
# alpha_target = 0.1
# lambda_hat = online_binary_search(model, val_loader, alpha_target, device=DEVICE)
# print(f"λ̂ obtained: {lambda_hat:.4f}")